In this exercise, you will code a regression model from scratch, to predict housing prices in California. 

Each record in the data (x) is comprised of 9 fields which include the size of the apartment in $m^2$, number of rooms, the location, etc.

The data is split into a training and test set. There are 8 features for one data entry (8 columns for one row) and the targets (labels) are the median values of the houses at a location in $k$.

The model will be a linear regression model: 
$f_w (x) = w_1 x_1 + ... + w_{8} x_{8} + b$ 
where w,b are the model parameters we want to learn using gradient descent. 

The data can be downloaded by executing the already given cell below. You can find more information about the dataset on https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset .

Steps: 

1. Implement a function to compute the loss: loss = (prediction - y)$^2$. Note that this function should calculate the average loss over all training samples. In short: implement a function which calculates the MSE-loss.
2. Plot the predictions over the ground truth using `matplotlib`. Label the x and y axis appropriately and show the individual points **without** connecting them with lines.
3. Implement the body functions of the `__init__` and `forward` methods of `LinearRegressor` to hold the model parameters w and b (both initialised using a normal distribution) and implement its correct linear regression equation function. Instantiate your module and show its trainable parameters to verify that w and b are within them.
4.  Implement the body of the following function to perform one training step, given a torch module (which contains the variables of your model), the training data and labels, as well as the optimizer. Use your MSEloss function to compute the loss and back-propagate the gradients using `.backward()`. Update the model weights using `optimizer.step()`. Remember to reset the gradients before the step.
5. Implement the body of the function for training and evaluating given a number of training steps. After every 1000 training steps the function should optionally (i.e., controlled via the `verbose` flag) output the loss on the training set, as well as the test set. The function should return the test set loss on the **last** step.
6. Instantiate your model again and wrap it in a suitable stochastic gradient descent optimizer. Use a learning rate of 0.001. Run your function for training and evaluation for 10,000 training steps. At the end of it, show the trained model parameters. Compare the final loss to your baseline loss.
7. Plot the test set predictions over the test set ground truth, and additionally compute the loss and the Pearson correlation (see `sklearn` cell).
8. Do not feed the entire training set for every iteration (gradient descent), but rather a random subset of 32 training examples (this is called a `batch` and the optimization process `stochastic` gradient descent). Use the given function template and implement its body. Run the function for the same number of iterations and with the same learning rate. Show its loss and plot the predictions. Compare with your baseline and full-batch gradient descent. `Hint:` Implement without replacement, i.e., each sample is taken only once. In order to train a model from scratch, you need to re-instantiate it and wrap it in a new optimizer.
9. Benchmark full-batch gradient descent and stochastic gradient descent for a set of different learning rates ($[.1, .01, .001, .0001, .00001]$). Use 10k iterations. Print, plot, and compare the losses on the last step for each training algorithm and each learning rate. What do you observe? What does it mean? Find a proper way to present the information in the plots such that your takeaway message is clear!
10. Benchmark full-batch gradient descent and stochastic gradient descent for different iterations ($[10, 100, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]$). Keep the learning rate fixed to $0.0001$. Print, plot, and compare the loss on the last step for each training algorithm and the different number of iterations. What do you observe? What does it mean? Find a proper way to present the information in the plots such that your takeaway message is clear!
11. Fill in the class body to implement SGD without automatic differentiation. Do not use torch.nn.Parameter and instead implement a manual backward call. Fill in the new step function that does not use autograd but calls this custom backward call. The gradients of the two parameters can be computed by using the following formulas: $\delta w = 2 \frac{1}{N} \sum_{i=1}^{N}x_i (\hat{y}_i-y_i)$ and $\delta b = 2 \frac{1}{N} \sum_{i=1}^{N}(\hat{y}_i-y_i)$, where $N$ is the batch size and $\hat{y}_i$ the prediction for the $i^{th}$ instance. Test the implementation with the call to training already given. Plot the predictions on the test set and compute the MSELoss and Pearson R.

`Note:` Do not use loops for the backpropagation in a batch.

In [ ]:
# import necessary functions
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import scipy
import torch
import typing

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
print("numpy: " + np.__version__)
print("torch: " + torch.__version__)
print("sklearn: " + sklearn.__version__)
print("scipy: " + scipy.__version__)

We first load and standardise the features (x_train and x_test), such that the mean is 0 and the standard deviation is 1 for each of the 8 features, i.e. standardise each column. For this purpose, we use the following formula: $x_{norm} = \frac{x - \mu}{\sigma} $, where $\mu$ is the mean and $\sigma$ is the standard deviation. We also need to normalise the labels (y_train and y_test) in the range [0-1]. This can be done by applying the following formula: $y_{norm} = \frac{y-\min{y}}{\max{y}-\min{y}}$. The normalisation parameters for both features and labels are computed on the training set.

In [ ]:
# Download and load the data using sklearn
X, y = fetch_california_housing(return_X_y=True)

# Cast to 32 bits which is the default in torch
X = X.astype(np.float32)
y = y.astype(np.float32)

# Split the data into 80% training and 20% test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print("X_train shape: " + str(X_train.shape))
print("X_test shape: " + str(X_test.shape))
print("y_train shape: " + str(y_train.shape))
print("y_test shape: " + str(y_test.shape))

# Compute feature statistics on training set
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

# Normalise training set features
X_train = (X_train - mean) / std
print(f'Feature mean (train): {X_train.mean()}')
print(f'Feature std (train): {X_train.std()}')

# Normalise test set features
X_test = (X_test - mean) / std
print(f'Feature mean (test): {X_test.mean()}')
print(f'Feature std (test): {X_test.std()}')

# Compute label statistics on training set
maximum = y_train.max()
minimum = y_train.min()

# Normalise training set labels
y_train = (y_train - minimum) / (maximum-minimum)
print(f'Label max (train): {y_train.max()}')
print(f'Label min (train): {y_train.min()}')

# Normalise test set labels
y_test = (y_test - minimum) / (maximum-minimum)
print(f'Label max (test): {y_test.max()}')
print(f'Label min (test): {y_test.min()}')

# Convert everything from numpy arrays to tensors:
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)

X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)

We implement a `LinearRegression` baseline from `sklearn`. This should serve as your reference for what is expected by a state-of-the-art implementation. Your goal throughout this day is to get similar (or maybe even better!) results from the models you implement yourself.

In [ ]:
baseline = LinearRegression()
baseline.fit(
    X_train.numpy(), 
    y_train.numpy()
)
y_hat = baseline.predict(X_test.numpy())
R = scipy.stats.pearsonr(y_hat, y_test)[0]
print(f'Baseline Pearson R: {R}')

1. Implement a function to compute the loss: loss = (prediction - y)$^2$. Note that this function should calculate the average loss over all training samples. In short: implement a function which calculates the MSE-loss.

In [ ]:
def MSEloss(prediction, label):
    return ((prediction - label) ** 2).mean()

loss = MSEloss(torch.from_numpy(y_hat), y_test)
print(f'Baseline MSE: {loss}')

2. Plot the predictions over the ground truth using `matplotlib`. Label the x and y axis appropriately and show the individual points **without** connecting them with lines.

In [ ]:
# 2. Plot the predictions over the ground truth using `matplotlib`. Label the x and y axis appropriately and show the individual points **without** connecting them with lines.
plt.scatter(y_test, y_hat)
plt.xlabel('Ground truth')
plt.ylabel('Predictions')
plt.title('Predictions vs Ground truth')
plt.show()

3. Implement the body functions of the `__init__` and `forward` methods of `LinearRegressor` to hold the model parameters w and b (both initialised using a normal distribution) and implement its correct linear regression equation function (using a matrix multiplication). Instantiate your module and show its trainable parameters to verify that w and b are within them.

In [ ]:
class LinearRegressor(torch.nn.Module):
    r"""Class implementing linear regression model.
    
    Args:
        num_features: number of input features
        num_outputs: number of output labels
    """
    def __init__(self, num_features: int, num_outputs: int = 1):
        super(LinearRegressor, self).__init__()
        self.w = torch.nn.Parameter(torch.randn(num_features, num_outputs)) # [num_features, num_outputs]
        self.b = torch.nn.Parameter(torch.randn(num_outputs)) # [num_outputs]
        
    def forward(self, x): # [batch, num_features] -> [batch, num_outputs]
        return torch.mm(x, self.w) + self.b
    
    def __repr__(self):
        return f'LinearRegressor(w: {self.w.shape}, b: {self.b.shape})'
    
# instantiate the model and show its trainable parameters
model = LinearRegressor(num_features=X_train.shape[1], num_outputs=1)
for name, param in model.named_parameters():
    print(f'{name}: {param.shape} requires_grad={param.requires_grad}')

4.  Implement the body of the following function to perform one training step, given a torch module (which contains the variables of your model), the training data and labels, as well as the optimizer. Use your MSEloss function to compute the loss and back-propagate the gradients using `.backward()`. Update the model weights using `optimizer.step()`. Remember to reset the gradients before the step.

In [ ]:
def autograd_train_step(
    model: torch.nn.Module, 
    optimizer: torch.optim.Optimizer, 
    X: torch.Tensor, 
    y: torch.Tensor
) -> torch.Tensor:
    r"""Single autograd training step.
    
    Function should use MSEloss defined above to compute loss,
    then perform a single optimization step to update model weights,
    and finally return the resulting loss.
    
    Args:
        model: pytorch model to be trained
        optimizer: optimizer wrapping pytorch model
        X: pytorch tensor containing features
        y: pytorch tensor containing labels
    Returns:
        torch.Tensor: loss at current step
    """
    y_hat = model(X)
    loss = MSEloss(y_hat, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss # training loss

5. Implement the body of the function for training and evaluating given a number of training steps. After every 1000 training steps the function should optionally (i.e., controlled via the `verbose` flag) output the loss on the training set, as well as the test set. The function should return the test set loss on the **last** step.

In [ ]:
def train_and_evaluate(
    model: torch.nn.Module, 
    optimizer: torch.optim.Optimizer, 
    X_train: torch.Tensor, 
    y_train: torch.Tensor, 
    X_test: torch.Tensor, 
    y_test: torch.Tensor, 
    iterations: int, 
    train_step: typing.Callable,
    verbose: bool = False
) -> torch.Tensor:
    r"""Train and evaluate model.
    
    Args:
        model: pytorch model to be trained
        optimizer: optimizer wrapping pytorch model
        X_train: pytorch tensor containing training features
        y_train: pytorch tensor containing training labels
        X_test: pytorch tensor containing test features
        y_test: pytorch tensor containing test labels
        iterations: number of iterations for gradient descent
        train_step: function implementing single training step
        verbose: flag controlling whether output should be printed at every iteration

    Returns:
        torch.Tensor: loss function at the last step
    """

    for i in range(iterations):
        train_loss = train_step(model, optimizer, X_train, y_train)
        if verbose and i % 1000 == 999:
            test_loss = MSEloss(model(X_test), y_test)
            print(f'Iteration {i+1}, train_loss: {train_loss} test_loss: {test_loss}')
    
    test_loss = MSEloss(model(X_test), y_test)
    return test_loss

6. Instantiate your model again and wrap it in a suitable stochastic gradient descent optimizer. Use a learning rate of 0.001. Run your function for training and evaluation for 10,000 training steps. At the end of it, show the trained model parameters. Compare the final loss to your baseline loss.

In [ ]:
model = LinearRegressor(num_features=X_train.shape[1], num_outputs=1)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
test_loss = train_and_evaluate(
    model, 
    optimizer, 
    X_train, 
    y_train, 
    X_test, 
    y_test, 
    iterations=10_000, 
    train_step=autograd_train_step,
    verbose=True
)

# show trained parameters
for name, param in model.named_parameters():
    print(f'{name}: {param.data}')

# compare final loss to baseline loss
print(f'Final test MSE loss: {test_loss}')
print(f'Baseline test MSE loss: {loss}')

7. Plot the test set predictions over the test set ground truth, and additionally compute the loss and the Pearson correlation (see `sklearn` cell).

8. Do not feed the entire training set for every iteration (gradient descent), but rather a random subset of 32 training examples (this is called a `batch` and the optimization process stochastic (or rather mini-batch) gradient descent). Use the given function template and implement its body. Run the function for the same number of iterations and with the same learning rate. Show its loss and plot the predictions. Compare with your baseline and full-batch gradient descent. In order to train a model from scratch, you need to re-instantiate it and wrap it in a new optimizer.

In [ ]:
def train_and_evaluate_on_batches(
    model: torch.nn.Module, 
    optimizer: torch.optim.Optimizer, 
    X_train: torch.Tensor, 
    y_train: torch.Tensor, 
    X_test: torch.Tensor, 
    y_test: torch.Tensor, 
    iterations: int,
    batch_size: int,
    train_step: typing.Callable,
    verbose: bool = False
):
    r"""Train and evaluate model on batches.
    
    Args:
        model: pytorch model to be trained
        optimizer: optimizer wrapping pytorch model
        X_train: pytorch tensor containing training features
        y_train: pytorch tensor containing training labels
        X_test: pytorch tensor containing test features
        y_test: pytorch tensor containing test labels
        iterations: number of iterations for gradient descent
        train_step: function implementing single training step
        verbose: flag controlling whether output should be printed at every iteration

    Returns:
        torch.Tensor: loss function at the last step
    """
    raise NotImplementedError

9. Benchmark full-batch gradient descent and stochastic gradient descent for a set of different learning rates ($[.1, .01, .001, .0001, .00001]$). Use 10k iterations. Print, plot, and compare the losses on the last step for each training algorithm and each learning rate. What do you observe? What does it mean? Find a proper way to present the information in the plots such that your takeaway message is clear!

10. Benchmark full-batch gradient descent and stochastic gradient descent for different iterations ($[10, 100, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]$). Keep the learning rate fixed to $0.0001$. Print, plot, and compare the loss on the last step for each training algorithm and the different number of iterations. What do you observe? What does it mean? Find a proper way to present the information in the plots such that your takeaway message is clear!

11. Fill in the class body to implement SGD without automatic differentiation. Do not use torch.nn.Parameter and instead implement a manual backward call. Fill in the new step function that does not use autograd but calls this custom backward call. The gradients of the two parameters can be computed by using the following formulas: $\delta w = 2 \frac{1}{N} \sum_{i=1}^{N}x_i (\hat{y}_i-y_i)$ and $\delta b = 2 \frac{1}{N} \sum_{i=1}^{N}(\hat{y}_i-y_i)$, where $N$ is the batch size and $\hat{y}_i$ the prediction for the $i^{th}$ instance. Test the implementation with the call to training already given. Plot the predictions on the test set and compute the MSELoss and Pearson R.
`Note:` Do not use loops for the backpropagation in a batch.

In [ ]:
class CustomLinearRegressor:
    """Custom linear regression class.
    
    Args:
        num_features: number of features in your linear regression
        num_outputs: number of outputs. Default: 1
        lr: learning rate for gradient descent
        
    """
    def __init__(self, num_features: int, num_outputs: int = 1, lr: float = 0.001):
        raise NotImplementedError
        
    def __call__(self, x):
        """Forward call function.
        
        Implement linear regression.
        
        Args:
            x: data
        """
        raise NotImplementedError
    
    def gradient_descent(self, x, y, y_hat):
        """Backward call function.
        
        Implement parameter update
        
        Args:
            x: data
            y: ground truth labels
            y_hat: model predictions
        """
        raise NotImplementedError
    
    def __repr__(self):
        return f'CustomLinearRegressor(w: {self.w.shape}, b: {self.b.shape})'

custom_model = CustomLinearRegressor(8)
print(custom_model)

def custom_train_step(
    model: torch.nn.Module, 
    optimizer: torch.optim.Optimizer, 
    X_train: torch.Tensor, 
    y_train: torch.Tensor
):
    r"""Function to implement training with custom differentiation.
    
    Args:
        model: pytorch model to be trained with custom differentiation
        optimizer: parameter provided for compatibility and should be IGNORED.
        X: pytorch tensor containing features
        y: pytorch tensor containing labels
    """
    pred = model(X_train)
    model.gradient_descent(X_train, y_train, pred)
    loss = MSEloss(pred, y_train)
    return loss

iterations = 10000
loss = train_and_evaluate_on_batches(
    model=custom_model, 
    optimizer=None, 
    X_train=X_train, 
    y_train=y_train, 
    X_test=X_test, 
    y_test=y_test, 
    iterations=iterations, 
    batch_size=batch_size,
    train_step=custom_train_step
)